In [1]:
import warnings
warnings.filterwarnings('ignore')

import api_keys


In [2]:
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field

In [210]:
from langchain_core.messages import SystemMessage

class Profile(BaseModel):
    character_name: str
    universe: str
    requirements: str
    user_name: str

profile_system_prompt = '''Your role is to become a character who engages in conversation with the user.
To do this, you should collect the following information from the user:

- What the character's name is
- What universe(세계관, 영화, 게임 등) does the character belong to
- What the user's requirements are
- Whtr the user's name is

If you cannot determine this information, ask the user directly to clarify — preferably using a bullet-point or structured format. Do not make assumptions.
Once you have all the necessary information, confirm it with the user one more time, and then call the relevant tool.'''


llm  = ChatOpenAI(model="gpt-4o-mini")
profiling_llm = llm.bind_tools([Profile])

profiling_llm.invoke([("assistant", profile_system_prompt),("user", "character name:지우, universe: 포켓몬스터, requirements: 없음, my name is 여행자. 정보 확인완료. 바로 시작하자")])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nzNSD9B6lK9Gf9gzFTvjW9LP', 'function': {'arguments': '{"character_name":"지우","universe":"포켓몬스터","requirements":"없음","user_name":"여행자"}', 'name': 'Profile'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 208, 'total_tokens': 244, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BJbxJw76hsHrlTv1k4d9th8loKTTh', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6715f1a4-1ce2-4bb5-bc45-fc6948047339-0', tool_calls=[{'name': 'Profile', 'args': {'character_name': '지우', 'universe': '포켓몬스터', 'requirements': '없음', 'user_name': '여행자'}, 'id': 'call_nzNSD9B6lK9Gf9gzFTvjW9LP', 'type': 'tool_call'}], usage_me

### 웹 검색 툴

In [192]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)
web_search_tool.invoke({"query": "RAG란?"})

[{'title': '[Capstone #1] RAG : Retrieval-Augmented Generation - velog',
  'url': 'https://velog.io/@yun_haaaa/RAG-Retrieval-Augmented-Generation',
  'content': 'RAG란? RAG는 기존 모델 외부에서 관련 데이터를 검색하여 입력을 향상시키는 방법으로, 이를 통해 결과를 개선하며 더 풍부한 맥락을',
  'score': 0.89324445},
 {'title': '검색 증강 생성(RAG)이란? | 포괄적인 RAG 안내서 - Elastic',
  'url': 'https://www.elastic.co/kr/what-is/retrieval-augmented-generation',
  'content': 'RAG(검색 증강 생성)란 무엇인가?\nRAG 기본 요소를 넘어서\n검색 증강 생성(RAG)의 정의\n검색 증강 생성(RAG)은 프라이빗 또는 독점 데이터 소스의 정보로 텍스트 생성을 보완하는 기술입니다. 대규모 데이터 세트 또는 지식 기반을 검색하도록 설계된 검색 모델에 해당 정보를 가져와 읽을 수 있는 텍스트 응답을 생성하는 대규모 언어 모델(LLM)과 같은 생성 모델을 결합합니다.\n검색 증강 생성은 추가 데이터 소스의 컨텍스트를 더하고 훈련을 통해 LLM의 원래 지식 기반을 보완함으로써 검색 경험의 정확도를 개선할 수 있습니다. 따라서 모델을 다시 훈련할 필요 없이 대규모 언어 모델의 출력이 향상됩니다. 추가 정보 소스는 LLM의 훈련에 사용되지 않은 인터넷의 새로운 정보부터 독점 비즈니스 컨텍스트 또는 비즈니스에 속한 기밀 내부 문서에 이르기까지 다양합니다. [...] RAG는 생성형 AI 시스템이 외부 정보 소스를 사용하여 보다 정확한 상황 인식 응답을 생성할 수 있도록 해주기 때문에 질문 답변 및 콘텐츠 생성과 같은 작업에 유용합니다. 일반적으로 시맨틱 검색이나 하이브리드 검색과 같은 검색 방법을 구현하여 사용자 의도에 응답하

### Info 검색 쿼리

In [193]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [203]:
info_search_system_prompt = """Your role is to investigate character information based on the details provided by the user.
Given a character's name and universe, generate the most effective and natural web search query to gather information about the character's background, personality, and Character's says.

You MUST output the search query only."""

info_web_search_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", info_search_system_prompt),
        ("user", "{question}")
    ]
)

info_web_search_chain = info_web_search_prompt | llm | StrOutputParser()

In [204]:
web_query = info_web_search_chain.invoke({"question": {"name": "지우", "universe": "포켓몬스터", "requirements": "없음"}})
print(web_query)
web_search_tool.invoke(web_query)

"지우 포켓몬스터 캐릭터 배경 성격 대사"


### Vector DB 및 Retriever 선언

In [169]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

In [173]:
search_result = web_search_tool.invoke({"query": "RAG란?"})

embd = OpenAIEmbeddings()

vectorstore = Chroma(
    collection_name="rag-chroma",
    embedding_function=embd
)

retriever = vectorstore.as_retriever()

docs = [Document(page_content=result["content"]) for result in search_result]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=600, chunk_overlap=100
)

doc_splits = text_splitter.split_documents(docs)

vectorstore.add_documents(doc_splits)

retriever.invoke("RAG에서 학습이 진행되나요?")

[Document(metadata={}, page_content='2023년은 ChatGPT, Llama-2와 같은 기초적인 대형 언어 모델(LLM)들에 관한 해였다면, 2024년은 RAG (Retrieval Augmented Generation; 검색 증강 생성) 와 AI Agent(AI 에이전트)의 해가 될 것이라고 다수의 필드 전문가들은 예측합니다.\n이번 블로그 포스트에서는 2024년에 RAG가 주목 받는 이유, 작동 방식, 그리고 파인튜닝 방식과의 비교 등 RAG의 기초 상식을 살펴보도록 하겠습니다.\n📖 목차\n• RAG 란?\n• RAG가 주목 받는 이유\n• RAG의 장점 3가지\n• RAG 작동 방식\n• RAG vs. Fine-tuning(파인튜닝)\n• RAG의 미래 동향\n\u200d\n📍RAG란?'),
 Document(metadata={}, page_content='2023년은 ChatGPT, Llama-2와 같은 기초적인 대형 언어 모델(LLM)들에 관한 해였다면, 2024년은 RAG (Retrieval Augmented Generation; 검색 증강 생성) 와 AI Agent(AI 에이전트)의 해가 될 것이라고 다수의 필드 전문가들은 예측합니다.\n이번 블로그 포스트에서는 2024년에 RAG가 주목 받는 이유, 작동 방식, 그리고 파인튜닝 방식과의 비교 등 RAG의 기초 상식을 살펴보도록 하겠습니다.\n📖 목차\n• RAG 란?\n• RAG가 주목 받는 이유\n• RAG의 장점 3가지\n• RAG 작동 방식\n• RAG vs. Fine-tuning(파인튜닝)\n• RAG의 미래 동향\n\u200d\n📍RAG란?'),
 Document(metadata={}, page_content='RAG(Retrieval-Augmented Generation)는 대규모 언어 모델의 출력을 최적화하여 응답을 생성하기 전에 학습 데이터 소스 외부의 신뢰할 수 있는 지식 베이스를 참조하도록 하는 프로세스입니다. 대규모 언어 모델(LLM)은 방대한 양의

### Retrieval 평가 Chain 정의

In [23]:

class EvalDocuments(BaseModel):
    """
    Document evaluation class.
    The decision attribute can have a value of 'yes' or 'no' indicating the relevance of the document to the message.
    """
    decision: str = Field(description="Documents are relevant to message. This attribute can have a value of 'yes' or 'no'")
    

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
retrieval_llm_evaluator = llm.with_structured_output(EvalDocuments)

retrieval_eval_system = """You are an evaluator responsible for assessing whether a retrieved document is relevant to the user's message.
The user is having a conversation with a character. If the document contains related keywords or is semantically connected to the user's message, evaluate it as relevant.
Your goal is to filter out incorrectly retrieved documents.
If the user's message is related to the document, output 'yes'; otherwise, output 'no'."""

retrieval_eval_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", retrieval_eval_system),
        ("user", "Character Profile and User Name : {profile}\n\nRetrieved Document: {document}\n\nMessage: {message}")
    ]
)

retrieval_evaluator = retrieval_eval_prompt | retrieval_llm_evaluator

character_name = "지우"
universe = "포켓몬스터"
doc_text = "지우의 파트너는 피카츄이다."
message = "너의 파트너는 피카츄야."

print(retrieval_evaluator.invoke({"character_name":character_name, "universe":universe, "document":doc_text, "message":message}))

decision='yes'


### 필요정보 웹 검색 쿼리 작성 chain 정의

In [119]:
web_query_gen_system_prompt = """You are role-playing with user and acting as a given character. To respond to the user's messages, you need to perform web searches.
Your role is to generate an appropriate web search query to obtain the knowledge necessary to answer the user's messages.
Output the web search query — do not output anything else."""

web_query_gen_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", web_query_gen_system_prompt),
        ("user", "Character Profile and User Name : {profile}\n\n  User's Message: {message}")
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini")
web_search_chain = web_query_gen_prompt | llm | StrOutputParser()

In [ ]:
query = web_search_chain.invoke({"character_name": "지우", "universe": "피카츄", "message": "처음 포켓몬을 받은 마을은 어디였지?"})
web_search_tool.invoke(query)

[{'title': '피카츄 - 위키백과, 우리 모두의 백과사전',
  'url': 'https://ko.wikipedia.org/wiki/%ED%94%BC%EC%B9%B4%EC%B8%84',
  'content': "한지우와 피카츄의 만남을 그리는 애니메이션의 제1화의 줄거리는 이와 같다. 주인공 한지우는 태초마을의 나어린 소년으로서 올해 10살이 되어 포켓몬을 가질 자격이 주어졌다. 포켓몬 마스터가 될 생각에 들떠 꿈속에서마저 몬스터볼을 던지는 꿈을 꾸었는데 그게 자명종을 망가뜨리는 짓이 되고 만다. 지우는 급히 오박사의 연구소로 달려갔지만 스타팅 포켓몬은 다 주어지고 없는 상태였다. 실망한 지우에게 오박사는 포켓몬이 한 마리 남아 있다고 알려줬는데 바로 수컷 피카츄였다. 다만 이 피카츄는 성격이 사나워서 지우의 말을 따르지 않고 툭하면 전기공격을 먹이며 포켓몬을 담아 편하게 옮길 수 있는 몬스터볼에도 갑갑하다면서 들어가지 않으려는 것이었다. 이렇게 생각이 서로 맞지 않던 둘이었지만, 깨비참 떼에 함께 쫓기고 깨비참의 몰매를 맞던 것을 지우가 몸을 던져 구해주고[45] 자신을 성의있게 대해주면서 피카츄도 마음을 돌리게 된다. 그리하여 서로의 마음속에는 우정이 싹텄지만, 여전히 피카츄는 [...] 피카츄(일본어: ピカチュウ 피카추[*][주 1] 문화어: 삐까쮸)는 《포켓몬스터》에 등장하는 가상의 생명체이다. 애니메이션과 비디오 게임에서는 오타니 이쿠에가 그 목소리를 맡고 있다. 귀여운 전기 포켓몬을 그려달라는 스기모리 켄의 주문을 받아 니시다 아츠코가 디자인하였다.[1] 게임 프리크와 닌텐도가 만든 1996년 일본 비디오 게임 《포켓몬스터 레드·그린》에 처음 등장했으며 1998년 《포켓몬스터 레드·블루》로 미국에 출시되었다. 피카츄는 전기 능력을 가진 노란색 쥐처럼 생긴 생물이다. 피카츄는 포켓몬 프랜차이즈의 주요 캐릭터이며, 마스코트이자 닌텐도의 주요 마스코트 역할을 한다. [...] 피카츄는 가장 인기 있고 잘 알려진 1세대 포켓몬인데, 포켓몬스터 TV 

### 답변 작성 Chain 정의하기

In [28]:
from langchain import hub

# rag_prompt = hub.pull("rlm/rag-prompt")

llm = ChatOpenAI(model="gpt-4o-mini")

rag_system_prompt = """You are a character engaged in a roleplay with the user.
Respond to the user's messages based on the pieces of retrieved context provided.
If the context is not needed, you may answer without using it.
If you're asked something you don't know, simply say you don't know.
Pay close attention to the context of the conversation provided by the user, and respond in a way that stays true to the profile of the character you are roleplaying.
Always follow up your response with an appropriate question to keep the conversation going."""

rag_user_prompt = """Character Profile and User Name : {profile}

Context: {context}

Conversations: {messages}

Answer:"""

rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rag_system_prompt),
        ("user", rag_user_prompt)
    ]
)

rag_chain = rag_prompt | llm | StrOutputParser()

### Eval Answer

In [159]:
from langchain_core.messages import AIMessage, HumanMessage

def EvalResponse(BaseModel):
    """
    Response evaluation class.
    The decision attribute can have a value of 'yes' or 'no' indicating the relevance of the response to the conversation.
    """
    decision: str = Field(description="Response is relevant to message. This attribute can have a value of 'yes' or 'no'")
    
    
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
response_llm_evaluator = llm.with_structured_output(EvalResponse)

response_eval_system = """You are a character engaged in a roleplay with the user.
Your role is to evaluate whether a response is appropriate to the conversation based on the character's profile and context.
Determine whether your response is appropriate to the conversation.
If your response properly addresses the issue or question in the conversation, output 'yes'; otherwise, output 'no'."""



response_eval_user_prompt = """Character Profile and User Name : {profile}
Context: {context}

Conversations: {messages}

Your Response: {response}

Decision:"""

response_eval_prompt = ChatPromptTemplate(
    [
        ("system", response_eval_system),
        ("user", response_eval_user_prompt)
    ]
)

response_eval_chain = response_eval_prompt | response_llm_evaluator

### Rewrite Query

In [71]:
llm = ChatOpenAI(model="gpt-4o-mini")

# Prompt
system = """You are a question rewriter that refines input queries to enhance their effectiveness for vector store retrieval or web searching by capturing their underlying semantic intent."""

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("user","Here is the original question:\n{question}\n\nPlease rewrite it to improve clarity and optimize it for retrieval."),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": "RAG에 대해서 알려주세요"})

'RAG의 개념과 기능에 대해 설명해 줄 수 있나요?'

### Define Nodes

In [72]:
#1. profiling_llm
#2. web_search_tool
#3. info_web_search_chain
#4. vectorstore
#5. retriever
#6. retrieval_evaluator
#7. web_search_chain
#8. rag_chain
#9. response_eval_chain
#10. question_rewriter


In [219]:
from typing import Annotated
from langgraph.graph.message import add_messages

def ConversationState(TypedDict):
    messages: Annotated[list, add_messages]
    documents: Annotated[list, add_messages]
    web_query: Annotated[list, add_messages]
    user_message: str
    generation: str
    retries:int
    profile: dict

In [ ]:
def get_profile_messages(messages):
    return [SystemMessage(content=profile_system_prompt)] + messages

def profile_node(state):
    print("--- Profiling ---")
    messages = get_profile_messages(state["messages"])
    response = profiling_llm.invoke(messages)
    return {"messages": [response]}
    
#     if response.tool_calls: # Tool Call 발생한 경우
#         profile = response.tool_calls[0]["args"]
#         return {"messages": [response], "profile": profile}
#     else:
#         return {"messages": [response]}

# state = profile_node({"messages": [HumanMessage(content="대화하고 싶어")]})
# print(state)

from langgraph.graph import START, END

def route_message(state):
    print("--- Route Message ---")
    messages = state["messages"]
    profile = state.get("profile")
    
    
    if profile: # Profile 수집 완료 상태, 대화 시작
        return "retriever"
    elif isinstance(messages[-1], AIMessage) and messages[-1].tool_calls: # Tool Call 발생했을 경우 Profile 수집 완료, "profile_web_search"로 라우팅
        return "profile_web_search"
    else: # 사용자에게 재질문하기 위해 Graph Escape
        return END
    
    
def profile_web_search(state):
    print("--- Profile Web Search ---")
    messages = state["messages"]
    profile = messages.tool_calls[0]["args"]
    
    
    return 